In [1]:
import numpy as np
import cv2

## The Summary of SqueezeNet

In [8]:
def squeezenet_summary(net, layer_type=False, layer_shape=False, 
                       get_flops=False, get_memory=False,
                       get_total_time=False, get_layer_time=False):
    # check if the model is empty
    if net.empty():
        print("Net is empty")
        return
        
    # get the types of layers used in the model
    if layer_type:
        print("\nNet contains:")
        for t in net.getLayerTypes():
            print("\t{} layers of type {}".format(net.getLayersCount(t), t))
        
    
    
    # get the tensor shapes for the loaded model and specified input shape
    if layer_shape:
        layers_ids, in_shapes, out_shapes = net.getLayersShapes([1, 3, 224, 224])
        layers_names = net.getLayerNames()
        print("\nNet layers shapes: ")
        for l in range(len(layers_names)):
            in_num, out_num = len(in_shapes[l]), len(out_shapes[l])
            print("  Layer {} has {} inputs and {} outputs".format(layers_names[l],
                                                                 in_num, out_num))
            for i in range(in_num):
                print("\tinput #{} has shape {}".format(i, in_shapes[l][i].flatten()))
            for i in range(out_num):
                print("\toutput #{} has shape {}".format(i, out_shapes[l][i].flatten()))
            
    # compute the number of FLOPs
    if get_flops:
        print("\ngflops: ", net.getFLOPS((1, 3, 224, 224)) * 1e-9)
    
    # report the amount of memory consumed for storing weights and intermediate tensors
    if get_memory:
        w, b = net.getMemoryConsumption((1, 3, 224, 224))
        print("\nweights (mb):", w * 1e-6, ", blobs (mb):", b * 1e-6)
    
    # perform a forward pass for a mock input
    blob = cv2.dnn.blobFromImage(np.zeros((224, 224, 3), np.uint8), 1, (224, 224))
    net.setInput(blob)
    net.forward()
    
    # report the total time
    if get_total_time:
        total, timings = net.getPerfProfile()
        tick2ms = 1000 / cv2.getTickFrequency()
        print("\ninference (ms): {:2f}".format(total * tick2ms))
    
    # report the per layer inference time
    if get_layer_time:
        layer_names = net.getLayerNames()
        print("\n{: <30} {}".format("LAYER", "TIME (ms)"))
        for (i,t) in enumerate(timings):
            print("{: <30} {:.2f}".format(layer_names[i], t[0] * tick2ms))

In [9]:
# read the model
model_path = "./model/squeezenet_v1.1.caffemodel"
proto_path = "./model/squeezenet_v1.1.prototxt"
squeezenet = cv2.dnn.readNetFromCaffe(proto_path, model_path)

In [10]:
squeezenet_summary(squeezenet, layer_type=True)


Net contains:
	8 layers of type Concat
	26 layers of type Convolution
	1 layers of type Dropout
	1 layers of type Flatten
	4 layers of type Pooling
	26 layers of type ReLU
	1 layers of type Softmax
	1 layers of type __NetInputLayer__


In [13]:
squeezenet_summary(squeezenet, layer_shape=True)


Net layers shapes: 
  Layer conv1 has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1   3 224 224]
  Layer relu_conv1 has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1  64 111 111]
  Layer pool1 has 1 inputs and 1 outputs
	input #0 has shape [  1  64 111 111]
	output #0 has shape [  1  64 111 111]
  Layer fire2/squeeze1x1 has 1 inputs and 1 outputs
	input #0 has shape [  1  64 111 111]
	output #0 has shape [ 1 64 55 55]
  Layer fire2/relu_squeeze1x1 has 1 inputs and 1 outputs
	input #0 has shape [ 1 64 55 55]
	output #0 has shape [ 1 16 55 55]
  Layer fire2/expand1x1 has 1 inputs and 1 outputs
	input #0 has shape [ 1 16 55 55]
	output #0 has shape [ 1 16 55 55]
  Layer fire2/relu_expand1x1 has 1 inputs and 1 outputs
	input #0 has shape [ 1 16 55 55]
	output #0 has shape [ 1 64 55 55]
  Layer fire2/expand3x3 has 1 inputs and 1 outputs
	input #0 has shape [ 1 64 55 55]
	output #0 has shape [ 1 64 55 55]
  La

In [14]:
squeezenet_summary(squeezenet, get_flops=True, get_memory=True)


gflops:  0.7066281600000001

weights (mb): 4.941984 , blobs (mb): 28.797728


In [15]:
squeezenet_summary(squeezenet, get_total_time=True, get_layer_time=True)


inference (ms): 228.607952

LAYER                          TIME (ms)
conv1                          8.18
relu_conv1                     0.00
pool1                          3.11
fire2/squeeze1x1               1.94
fire2/relu_squeeze1x1          0.00
fire2/expand1x1                9.84
fire2/relu_expand1x1           0.00
fire2/expand3x3                16.31
fire2/relu_expand3x3           0.00
fire2/concat                   0.00
fire3/squeeze1x1               3.22
fire3/relu_squeeze1x1          0.00
fire3/expand1x1                41.52
fire3/relu_expand1x1           0.00
fire3/expand3x3                30.71
fire3/relu_expand3x3           0.00
fire3/concat                   0.00
pool3                          2.18
fire4/squeeze1x1               1.35
fire4/relu_squeeze1x1          0.00
fire4/expand1x1                1.72
fire4/relu_expand1x1           0.00
fire4/expand3x3                8.36
fire4/relu_expand3x3           0.00
fire4/concat                   0.00
fire5/squeeze1x1           